# Generative Adversarial Network (GAN) Model Trained on MNIST Dataset

In this notebook, we embark on the creation and training of a Generative Adversarial Network (GAN) using the MNIST dataset. The MNIST dataset is a widely used collection of 28x28 pixel grayscale images of handwritten digits (0 through 9). Each image is labeled with its corresponding digit, making it a popular choice for training and evaluating machine learning models.

## MNIST Dataset Overview

The MNIST dataset consists of 60,000 training images and 10,000 testing images, making it a benchmark dataset for digit recognition tasks. These images are widely employed for developing and testing various machine learning algorithms due to their simplicity and clarity.

## Objective of the Notebook

Our primary objective is to build a GAN capable of generating realistic-looking handwritten digits similar to those in the MNIST dataset. GANs are known for their ability to create synthetic data by training a generator to produce samples that are indistinguishable from real data, while a discriminator learns to differentiate between real and synthetic samples.

## Key Steps in the Notebook

1. **Loading and Preprocessing MNIST Data:** We'll start by loading the MNIST dataset and preparing it for training.

2. **Defining the Generator and Discriminator Models:** The notebook will guide you through the creation of the generator and discriminator models, crucial components of a GAN.

3. **Training the GAN:** We'll train the GAN on the MNIST dataset, allowing the generator to learn how to generate realistic digit images.

4. **Evaluating GAN Performance:** We'll assess the quality of the generated images and the overall performance of our GAN model.

By the end of this notebook, you'll have a better understanding of GANs and how they can be applied to generate new, authentic-looking digit images based on the MNIST dataset.

## Libraries
To install the required libraries, you can use the following `%pip` command in your Python environment:


This command will install the following libraries:

- **NumPy:** A powerful library for numerical operations in Python.
- **tqdm:** A fast, extensible progress bar for loops and iterables.
- **Matplotlib:** A comprehensive library for creating static, animated, and interactive visualizations in Python.
- **TensorFlow:** An open-source machine learning framework developed by Google for building and training machine learning models.


In [ ]:
%pip install numpy tdqm matplotlib tensorflow

## Implementation of code

In this code, we implement a Generative Adversarial Network (GAN) using the TensorFlow library. GANs consist of a generator and a discriminator trained simultaneously, where the generator creates synthetic data, and the discriminator distinguishes between real and synthetic data.

In [ ]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.datasets import mnist

# Define the generator model
def generator():
    model = Sequential()
    model.add(Dense(256, use_bias=False, input_shape=(100,)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(512, use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(784, activation='tanh', use_bias=False))
    return model

# Define the discriminator model
def discriminator():
    model = Sequential()
    model.add(Dense(256, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.25))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data(path='mnist.npz')

x_train = x_train.reshape(60000, 784)
x_train = x_train.astype('float32') / 255

# Create the generator and discriminator models
generator = generator()
discriminator = discriminator()

# Define the optimizers
generator_optimizer = Adam(lr=0.0002, beta_1=0.5, beta_2=0.999)
discriminator_optimizer = Adam(lr=0.0002, beta_1=0.5, beta_2=0.999)

# Define the loss functions
generator_loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
discriminator_loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)

generator.compile(optimizer=generator_optimizer, loss=generator_loss_fn)
discriminator.compile(optimizer=discriminator_optimizer, loss=discriminator_loss_fn, metrics=['accuracy'])



# Train the GAN
epochs = 50
batch_size = 128

for epoch in range(epochs):
    # Use tqdm to create a progress bar for the outer loop
    with tqdm(total=len(x_train), desc=f'Epoch {epoch + 1}/{epochs}', unit='img') as pbar:
        for i in range(0, len(x_train) - batch_size + 1, batch_size):
            real_images = x_train[i:i + batch_size]
            noise = np.random.normal(0, 1, (batch_size, 100))
            generated_images = generator.predict(noise)

            real_labels = np.ones((batch_size, 1))
            fake_labels = np.zeros((batch_size, 1))

            real_loss = discriminator.train_on_batch(real_images, real_labels)
            fake_loss = discriminator.train_on_batch(generated_images, fake_labels)
            discriminator_loss = 0.5 * np.add(real_loss, fake_loss)

            # Train the generator
            noise = np.random.normal(0, 1, (batch_size, 100))
            generated_labels = np.ones((batch_size, 1))

            with tf.GradientTape() as gen_tape:
                generated_images = generator(noise)
                fake_output = discriminator(generated_images)
                generator_loss = generator_loss_fn(generated_labels, fake_output)

            generator_gradients = gen_tape.gradient(generator_loss, generator.trainable_variables)

            # Update the generator weights
            generator.optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))

            # Update the progress bar
            pbar.update(batch_size)
            pbar.set_postfix({'Discriminator Loss': discriminator_loss, 'Generator Loss': generator_loss.numpy()})
            
# Save the generator model
generator.save('generator.h5')

# Save the discriminator model
discriminator.save('discriminator.h5')

## Generating Images with the Trained Model

Now that we've successfully trained our Generative Adversarial Network (GAN) model and saved the generator as 'generator.h5', let's explore how to generate synthetic images using this trained model.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Load the generator model
generator = load_model('generator.h5')

# Generate a batch of random noise
noise = np.random.normal(0, 1, (batch_size, 100))

# Generate images using the generator
generated_images = generator.predict(noise)

# Rescale the generated images to the range [0, 1]
generated_images = 0.5 * generated_images + 0.5

# Display the generated images
rows, cols = 4, 4  # Adjust as needed
fig, axs = plt.subplots(rows, cols)
fig.suptitle('Generated Images')
idx = 0
for i in range(rows):
    for j in range(cols):
        axs[i, j].imshow(generated_images[idx].reshape(28, 28), cmap='gray')
        axs[i, j].axis('off')
        idx += 1
plt.show()

## Generating and Evaluating Images with the Discriminator

In this section, our objective is to both generate synthetic images and assess their quality by leveraging the discriminator. The following code achieves the dual purpose of image generation and evaluation using the discriminator.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Load the generator and discriminator models
generator = load_model('generator.h5')
discriminator = load_model('discriminator.h5')

# Generate a batch of random noise
batch_size = 16  # Adjust as needed
noise = np.random.normal(0, 1, (batch_size, 100))

# Generate images using the generator
generated_images = generator.predict(noise)

# Rescale the generated images to the range [0, 1]
generated_images = 0.5 * generated_images + 0.5

# Display the generated images
rows, cols = 4, 4  # Adjust as needed
fig, axs = plt.subplots(rows, cols)
fig.suptitle('Generated Images')
idx = 0
for i in range(rows):
    for j in range(cols):
        axs[i, j].imshow(generated_images[idx].reshape(28, 28), cmap='gray')
        axs[i, j].axis('off')
        idx += 1
plt.show()


# Evaluate generated images using the discriminator
discriminator_predictions = discriminator.predict(generated_images)

# Print discriminator predictions for each generated image
for i in range(batch_size):
    print(f"Image {i + 1} - Discriminator Prediction: {discriminator_predictions[i][0]}")
